In [ ]:
# 새로운 픽셀 값을 얻을 때 하나의 픽셀 값이 아닌 그 주변 픽셀들의 값을 활용하는 방법을 공간 영역 필터링(spacial domain filtering)
#블러링(Blurring)이란 기존의 영상을 흐릿하게 만드는 작업
#예제6.1 박스 필터와 양방향 필터
import cv2
import numpy as np

src = cv2.imread('0205.png\image_2.jpg', cv2.IMREAD_GRAYSCALE)

dst1= cv2.boxFilter(src, ddepth = -1, ksize=(11,11))
dst2= cv2.boxFilter(src, ddepth = -1, ksize=(21,21))
#ddepth: 출력 영상의 dtype (-1: 입력 영상과 동일) / ksize: 커널의 크기
dst3= cv2.bilateralFilter(src, d = 11, sigmaColor= 10,
                          sigmaSpace= 10)
dst4= cv2.bilateralFilter(src, d = -1, sigmaColor= 10,
                          sigmaSpace= 10)
#잡음 제거+경계 뚜렷=>바이레터럴 필터는 이를 개선하기 위해 가우시안 필터와 경계 필터를 결합
#sigmaColor: 색공간의 시그마 값 / sigmaSpace: 좌표 공간의 시그마 값
cv2.imshow('dst1', dst1) 
cv2.imshow('dst2', dst2) 
cv2.imshow('dst3', dst3) 
cv2.imshow('dst4', dst4) 
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
#6.2 미디안 필터, 블러, 가우시안 필터
import cv2
import numpy as np

src = cv2.imread('0205.png\image_3.jpg', cv2.IMREAD_GRAYSCALE)

dst1= cv2.medianBlur(src, ksize= 7) #소금, 후추 잡음 제거
dst2= cv2.blur(src, ksize= (7,7))
dst3= cv2.GaussianBlur(src, ksize=(7,7), sigmaX= 0.0)
dst4= cv2.GaussianBlur(src, ksize=(7,7), sigmaX= 10.0)

cv2.imshow('dst1', dst1)
cv2.imshow('dst2', dst2) 
cv2.imshow('dst3', dst3) 
cv2.imshow('dst4', dst4) 
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
#에지는 영상에서 픽셀의 밝기 값이 급격하게 변하는 부분
#영상에 미분을 하는 이유는 에지를 검출하기 위함
#예제6.3 Solbel 필터1
import cv2
import numpy as np

src = cv2.imread('0205.png\dark_image.jpg', cv2.IMREAD_GRAYSCALE)
#1
gx = cv2.Sobel(src, cv2.CV_32F, 1, 0, ksize=3)
gy = cv2.Sobel(src, cv2.CV_32F, 0, 1, ksize=3)

#2
dstX = cv2.sqrt(np.abs(gx))
dstX = cv2.normalize(dstX, None, 0, 255, cv2.NORM_MINMAX,
                     dtype= cv2.CV_8U)

#3
dstY = cv2.sqrt(np.abs(gy))
dstY = cv2.normalize(dstY, None, 0, 255, cv2.NORM_MINMAX,
                     dtype= cv2.CV_8U)

#4
mag = cv2.magnitude(gx, gy) #그래디언트 크기 mag
minVal, maxVal, minLoc, maxLoc = cv2.minMaxLoc(mag) #배열 전역의 최댓값, 최솟값 찾기
print('mag:', minVal, minLoc, maxLoc)

dstM = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX,
                     dtype= cv2.CV_8U)

cv2.imshow('src', src)
cv2.imshow('dstX', dstX)
cv2.imshow('dstY', dstY)
cv2.imshow('dstM', dstM)
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
#예제 6.4 Sobel 필터 2:에지 그래디언트 방향
import cv2
import numpy as np
from matplotlib import pyplot as plt

src = cv2.imread('0205.png\\tiger.jpg', cv2.IMREAD_GRAYSCALE)
cv2.imshow('src', src)

#1
gx = cv2.Sobel(src, cv2.CV_32F, 1, 0, ksize=3)
gy = cv2.Sobel(src, cv2.CV_32F, 0, 1, ksize=3)

mag, angle = cv2.cartToPolar(gx, gy, angleInDegrees= True) #그래디언트 크기mag, 각도 angle
minVal, maxVal, minLoc, maxLoc = cv2.minMaxLoc(angle)
print('angle:', minVal, maxVal, minLoc, maxLoc)

#2 이진영상 edge 생성
ret, edge = cv2.threshold(mag, 100, 255, cv2.THRESH_BINARY)
edge = edge.astype(np.uint8)
cv2.imshow('edge', edge)

#3 
height, width = mag.shape[:2]
angleM = np.full((height, width, 3), (255, 255, 255), dtype= np.int8) #배경이 흰색(255, 255, 255)인 3-채널 컬러 영상 생성
for y in range(height):
    for x in range(width):
        if edge[y, x] != 0:     #if mag[y, x] > 100: # edge
            if angle[y, x] == 0:
                angleM[y, x] = (0, 0, 255)  #red
            elif angle[y, x] == 90:
                angleM[y, x] = (0, 255, 0)  #green
            elif angle[y, x] == 180:
                angleM[y, x] = (255, 0, 0)  #blue
            elif angle[y, x] == 270:
                angleM[y, x] = (0, 255, 255)#yellow
            else:
                angleM[y, x] = (128, 128, 128)  #gray
cv2.imshow('angleM', angleM)
##cv2.waitkey()
##cv2.destroyAllwindows()

#4
hist = cv2.calcHist(images=[angle], channels=[0], mask= edge,
                    histSize= [360], ranges= [0, 360])
    
hist = hist.flatten()
##plt.title('hist: binX = np.arange(360)')
plt.plot(hist, color = 'r')
binX = np.arange(360)
plt.bar(binX, hist, width = 1, color = 'b')
plt.show()

In [ ]:
#예제6.5 Laplacian 필터1
import cv2
import numpy as np

#1
src = cv2.imread('0205.png\dark_image.jpg', cv2.IMREAD_GRAYSCALE)
blur= cv2.GaussianBlur(src, ksize= (7,7), sigmaX= 0.0)
cv2.imshow('src', src)
cv2.imshow('blur', blur)

#2
lap = cv2.Laplacian(src, cv2.CV_32F)
minVal, maxVal, minLoc, maxLoc = cv2.minMaxLoc(lap)
print('lap:', minVal, maxVal, minLoc, maxLoc)
dst = cv2.convertScaleAbs(lap)
dst = cv2.normalize(dst, None, 0, 255, cv2.NORM_MINMAX)
cv2.imshow('lap', lap)
cv2.imshow('dst', dst)

#3
lap2 = cv2.Laplacian(blur, cv2.CV_32F)
minVal, maxVal, minLoc, maxLoc = cv2.minMaxLoc(lap2)
print('lap2:', minVal, minLoc, maxVal, minLoc, maxLoc)
dst2 = cv2.convertScaleAbs(lap2)
dst2 = cv2.normalize(dst2, None, 0, 255, cv2.NORM_MINMAX)

cv2.imshow('lap2', lap2)
cv2.imshow('dst2', dst2)

cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
#예제6.6 Laplacian 필터2: 0-교차 zero-crossing
# 잡음에 덜 민감한 Laplacian Filter를 적용할 수 있다는 장점
import cv2
import numpy as np

#1
src = cv2.imread('0205.png\\rectangle2.jpg', cv2.IMREAD_GRAYSCALE)
blur= cv2.GaussianBlur(src, ksize= (7,7), sigmaX= 0.0)
lap = cv2.Laplacian(blur, cv2.CV_32F, 3)

ret, edge = cv2.threshold(np.abs(lap), 10, 255,
                           cv2.THRESH_BINARY)
edge = edge.astype(np.uint8)
cv2.imshow('edge', edge)

#2
def SGN(x):
    if x >= 0:
        sign = 1
    else:
        sign = -1
    return sign

def zeroCrossing(lap):
    height, width = lap.shape
    Z = np.zeros(lap.shape, dtype= np.uint8)
    for y in range(1, height - 1):
        for x in range(1, width - 1):
            neighbors=[lap[y - 1, x], lap[y + 1, x],
                       lap[y, x - 1], lap[y, x + 1],
                       lap[y - 1, x - 1], lap[y - 1, x + 1],
                       lap[y + 1, x - 1], lap[y + 1, x + 1]]
            mValue = min(neighbors)
            if SGN(lap[y, x]) != SGN(mValue):
                Z[y, x] = 255
    return Z
edgeZ = zeroCrossing(lap)
cv2.imshow('Zero Crossing', edgeZ)
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
#예제6.7 cv2.filter2D()와 cv2.sepFilter()에 의한 에지 검출
#cv2.filter2D : Kernel을 통하여 Image를 Filtering하는 방법
#cv2.sepFilter2D : image의 x, y를 각각의 kernel을 통하여 Filtering하는 방법
import cv2
import numpy as np

src = cv2.imread('0205.png\\tiger.jpg', cv2.IMREAD_GRAYSCALE)

#1
kx, ky = cv2.getDerivKernels(1, 0, ksize= 3)
sobelX = ky.dot(kx.T)
##sobelX = np.array([[-1, 0, 1],
##                   [-2, 0, 2],
##                   [-1, 0, 1]], dtype = np.float32)
print('kx=', kx)
print('ky=', ky)
print('sobelX', sobelX)
gx = cv2.filter2D(src, cv2.CV_32F, sobelX)
##gx = cv2.sepFilter2D(src, cv2.CV_32F, kx, ky)

#2
kx, ky = cv2.getDerivKernels(0, 1, ksize= 3)
sobelY = ky.dot(kx.T)
##sobelY = np.array([[-1, -2, -1],
##                   [ 0,  0,  0],
##                   [ 1,  2,  1]], dtype = np.float32)
print('kx=', kx)
print('ky=', ky)
print('sobelY', sobelY)
gy = cv2.filter2D(src, cv2.CV_32F, sobelY)
##gy = cv2.sepFilter2D(src, cv2.CV_32F, kx, ky)

#3
mag = cv2.magnitude(gx, gy)
ret, edge = cv2.threshold(mag, 100, 255, cv2.THRESH_BINARY)

cv2.imshow('edge', edge)
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
#예제6.8 : LoG 필터링, 0-교차 에지 영상
#Log(Laplacian of Gaussian) : Gausian Filter를 통하여 Noise제거후 Laplacian Filter를 적용-> 한개의 필터로 선언
import cv2
import numpy as np

src = cv2.imread('0205.png\\tiger.jpg', cv2.IMREAD_GRAYSCALE)

#1
def logFilter(ksize = 7):
    k2 = ksize//2
    sigma = 0.3 * (k2-1)+0.8
    print('sigma=',sigma)
    LoG = np.zeros((ksize,ksize),dtype=np.float32)
    for y in range(-k2,k2+1):
        for x in range(-k2,k2+1):
            g = -(x*x+y*y)/(2.0*sigma**2.0)
            LoG[y+k2,x+k2] = -(1.0+g)*np.exp(g)/(np.pi*sigma**4.0)
    
    return LoG

#2
kernel = logFilter()            #7, 15, 31, 51
LoG = cv2.filter2D(src ,cv2.CV_32F,kernel)
cv2.imshow('LoG', LoG)

#3
def zeroCrossing2(lap,thresh = 0.01): #thresh 임계값
    height, width = lap.shape
    Z = np.zeros(lap.shape,dtype=np.uint8)
    for y in range(1, height - 1):
        for x in range(1, width - 1):
            neighbors = [lap[y - 1, x],lap[y + 1, x],
                         lap[y, x - 1],lap[y, x + 1],
                         lap[y - 1, x - 1],lap[y - 1, x + 1],
                         lap[y + 1, x - 1],lap[y + 1, x + 1]]
            pos = 0
            neg = 0
            for value in neighbors:
                if value > thresh:
                    pos += 1
                if value < thresh:      # value < thresh
                    neg += 1
            if pos > 0 and neg > 0:
                Z[y, x] = 255
    
    return Z
edgeZ = zeroCrossing2(LoG)
cv2.imshow('Zero Crossing2', edgeZ)
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
#예제6.9 erode()와 dilate() 모폴로지 연산1
import cv2
import numpy as np

src = cv2.imread('0205.png\\tiger.jpg', cv2.IMREAD_GRAYSCALE)
kernel = cv2.getStructuringElement(shape= cv2.MORPH_RECT, #MORPH_RECT: 사각형
                                   ksize= (3,3)) #ksize: structuring element 사이즈
erode = cv2.erode(src, kernel, iterations= 5) #erode: 연산결과 최소값 pixel인 검은색(0)이 점점커지거나 흰색이 제거되는것을 확인할 수 있다
dilate = cv2.dilate(src, kernel, iterations= 5) #dilate: 연산결과 최댓값 pixel인 흰색(255)이 점점커지거나 검은색이 제거되는것을 확인할 수 있다.
erode2 = cv2.erode(dilate,kernel,iterations= 7)
##dilate2= cv2.dilate(erode2,kernel,iterations = 2)

cv2.imshow('src', src)
cv2.imshow('erode', erode)
cv2.imshow('dilate', dilate)
cv2.imshow('erode2', erode2)
##cv2.imshow('dilate2', dilate2)
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
#예제6.10 morphologyEx() 모폴로지 연산
import cv2
import numpy as np

src = cv2.imread('0205.png\example 6.10.jpg', cv2.IMREAD_GRAYSCALE)
kernel = cv2.getStructuringElement(shape = cv2.MORPH_RECT,
                                   ksize = (3,3))
closing = cv2.morphologyEx(src, cv2.MORPH_CLOSE, kernel,
                           iterations = 5)
#closing: dilate 후 erode연산, 흰색 물체 속의 검은색 잡음 제거
opening = cv2.morphologyEx(closing, cv2.MORPH_OPEN, kernel,
                           iterations = 5)
#opening: erode 후 dilate를 연산, 검은물체 내에 흰색 잡음 제거
gradient = cv2.morphologyEx(opening,cv2.MORPH_GRADIENT,kernel)
#gradient = cv2.morphologyEx(opening, cv2.MORPH_GRADIENT, kernel,
#                            iterations = 5)
#gradient: dilate - erode, 최대값 - 최소값으로서 급격한 차이를 가지는 Edge추출
tophat = cv2.morphologyEx(src, cv2.MORPH_TOPHAT, kernel,
                         iterations=5)
#검은색 배경속의 흰색 점을 검출
balckhat = cv2.morphologyEx(src, cv2.MORPH_BLACKHAT, kernel,
                            iterations = 5)
#흰색 물체 속의 검은색 점을 검출
cv2.imshow('opening', opening)
cv2.imshow('closing', closing)
cv2.imshow('gradient', gradient)
cv2.imshow('tophat', tophat)
cv2.imshow('balckhat', balckhat)
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
#예제6.11 모폴로지 연산 골격화skeleton
import cv2
import numpy as np

#1
src = cv2.imread('0205.png\T.jpg', cv2.IMREAD_GRAYSCALE)
##src = cv2.bitwise_not(src) #mask영역에서의 보색 출력

ret, A = cv2.threshold(src, 128, 255, cv2.THRESH_BINARY)
skel_dst = np.zeros(src.shape, np.uint8)

shape1 = cv2.MORPH_CROSS #MORPH_RECT: 사각형
shape2 = cv2.MORPH_RECT  #MORPH_CROSS: 십자 모양

B = cv2.getStructuringElement(shape= shape1, ksize= (3,3))
done = True
while done:
    erode = cv2.erode(A, B) #A를 B로 erode침식
##    opening = cv2.dilate(erode, B)
    opening = cv2.morphologyEx(erode, cv2.MORPH_OPEN, B)    #erode를 B로 opening 열기
    # cv2.absdiff(erode, opening)
    tmp = cv2.subtract(erode, opening) #영상 뺄셈 연산
    skel_dst = cv2.bitwise_or(skel_dst, tmp) #mask 영역의 두 이미지를 합쳐버림
    A = erode.copy()
    done = cv2.countNonZero(A) != 0

##  cv2.imshow('opening', opening)
##  cv2.imshow('tmp', tmp)
##  cv2.imshow('skel_dst', skel_dst)
##  cv2.waitkey()

cv2.imshow('src', src)
cv2.imshow('skel_dst', skel_dst)
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
#예제6.12 cv2.matchTenplate() 템플릿 매칭
#템플릿 매칭: 원본 이미지에서 템플릿 이미지와 일치하는 영역을 찾는 알고리즘
import cv2
import numpy as np

src = cv2.imread('0205.png\\alphabet.bmp',0)
tmp_A = cv2.imread('0205.png\A.bmp',0)
tmp_S = cv2.imread('0205.png\S.bmp',0)
tmp_b = cv2.imread('0205.png\\b.bmp',0)
dst = cv2.cvtColor(src, cv2.COLOR_GRAY2BGR) #출력 표시 영상

#1 # image: 매칭 받을 이미지, templ: 매칭할 이미지, method: 매칭 방법
R1 = cv2.matchTemplate(src,tmp_A,cv2.TM_SQDIFF_NORMED) #cv2.TM_SQDIFF를 정규화
minVal, _, minLoc, _ = cv2.minMaxLoc(R1) #최소 포인터, 최대 포인터, 최소 지점, 최대 지점
print('TM_SQDIFF_NORMED:', minVal, minLoc)

h, w = tmp_A.shape[:2]
cv2.rectangle(dst,minLoc,(minLoc[0] + w, minLoc[1] + h),
               (255,0,0), 2)

#2
R2 = cv2.matchTemplate(src, tmp_S, cv2.TM_CCORR_NORMED)
_, maxVal, _, maxLoc = cv2.minMaxLoc(R2)
print('TM_CCORR_NORMED:', maxVal, maxLoc)
h, w = tmp_S.shape[:2]
cv2.rectangle(dst, maxLoc,(maxLoc[0] + w, maxLoc[1] + h),
               (0, 255, 0), 2)

#3
R3 = cv2.matchTemplate(src, tmp_b, cv2.TM_CCOEFF_NORMED)
_, maxVal, _, maxLoc = cv2.minMaxLoc(R3)
print('TM_CCOEFF_NORMED:', maxVal, maxLoc)
h, w = tmp_S.shape[:2]
cv2.rectangle(dst, maxLoc,(maxLoc[0] + w, maxLoc[1] + h),
               (0, 0, 255), 2)

cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()